In [45]:
# from distance_request import *
from datetime import datetime
import googlemaps
import pandas as pd

In [46]:
# Read in call & ambulance files
calls_df = pd.read_csv('calls.csv', parse_dates=[3])
amb_df = pd.read_csv('ambulance_loc.csv')

# Select the most recent call and the available ambulances
current_call = calls_df.loc[calls_df.TIME == max(calls_df.TIME)]
available_amb = amb_df.loc[amb_df.AVAILABLE == 1]

In [50]:
available_amb.head()

,AMB_ID,LAT,LONG,AVAILABLE
0,amb_1,37.615223,-122.389977,1
1,amb_2,37.802400,-122.405800,1
2,amb_3,37.750238,-122.433703,1
4,amb_5,37.774929,-122.419416,1
6,amb_7,37.785413,-122.429383,1


In [152]:
current_call

,CALL_ID,LAT,LONG,TIME
0,call_1,37.788001,-122.407409,2017-12-01 23:05:00


In [55]:
# Get coordinates of most recent call and available ambulances
call_coord = [(current_call.LAT[0], current_call.LONG[0])]
amb_coord = [(row[2], row[3]) for row in available_amb.itertuples()]

# Use call time as proxy for departure time
dep_time = current_call.TIME[0]

In [56]:
# Get api key to make call
def get_api_key(filepath):
    with open(filepath) as f:
        content = f.readlines()
    # remove whitespace characters at the end of each line
    content = [x.strip() for x in content]
    key = content[0]
    return key

key = get_api_key('/Users/melaniecostello/Desktop/maps_distance_key.txt')

In [72]:
# Make API call & store results in obj 'result'
gmaps = googlemaps.Client(key=key)
result = gmaps.distance_matrix(amb_coord, call_coord, mode="driving", units="imperial", departure_time=dep_time)

In [153]:
result

{'destination_addresses': ['350 Post St, San Francisco, CA 94108, USA'],
 'origin_addresses': ['International Terminal Arrivals Level, San Francisco, CA 94128, USA',
  '39-95 Telegraph Hill Blvd, San Francisco, CA 94133, USA',
  '1400-1414 Castro St, San Francisco, CA 94114, USA',
  '8411 Market St, San Francisco, CA 94103, USA',
  '1682 Post St, San Francisco, CA 94115, USA',
  '4900 Marie P DeBartolo Way, Santa Clara, CA 95054, USA',
  '6 Broadway, San Francisco, CA 94111, USA',
  '150 3rd St, San Francisco, CA 94103, USA'],
 'rows': [{'elements': [{'distance': {'text': '13.8 mi', 'value': 22205},
     'duration': {'text': '25 mins', 'value': 1496},
     'duration_in_traffic': {'text': '24 mins', 'value': 1413},
     'status': 'OK'}]},
  {'elements': [{'distance': {'text': '1.6 mi', 'value': 2544},
     'duration': {'text': '14 mins', 'value': 851},
     'duration_in_traffic': {'text': '13 mins', 'value': 801},
     'status': 'OK'}]},
  {'elements': [{'distance': {'text': '4.1 mi', '

In [141]:
# Helper function to parse results
def parse_api_obj(result, available_amb):
    output_mat = pd.DataFrame()
    for idx, row in enumerate(result['rows']):
        row_mat = pd.DataFrame()
        mat = row['elements'][0]
        for key, val in mat.items():
            if key != 'status':
                df = pd.DataFrame.from_dict(val, orient='index')
                df = df.transpose()
                df.columns = [key + "_" + c for c in df.columns]
                if row_mat.empty:
                    row_mat = df
                else:
                    row_mat = pd.concat([row_mat, df], axis=1)
        if output_mat.empty:
            output_mat = row_mat
        else:
            output_mat = output_mat.append(row_mat)
    output_mat.index = [amb for amb in available_amb.AMB_ID]
    return output_mat

In [143]:
# Get data frame of available ambulances
options = parse_api_obj(result, available_amb)

In [154]:
options

,duration_value,duration_text,distance_value,distance_text,duration_in_traffic_value,duration_in_traffic_text
amb_1,1496,25 mins,22205,13.8 mi,1413,24 mins
amb_2,851,14 mins,2544,1.6 mi,801,13 mins
amb_3,1359,23 mins,6520,4.1 mi,1258,21 mins
amb_5,674,11 mins,2934,1.8 mi,636,11 mins
amb_7,494,8 mins,1951,1.2 mi,477,8 mins
amb_8,3400,57 mins,70986,44.1 mi,3144,52 mins
amb_9,627,10 mins,2243,1.4 mi,564,9 mins
amb_10,308,5 mins,953,0.6 mi,277,5 mins


In [155]:
# Select ambulance for response based on desired metric
chosen = options.loc[options.duration_in_traffic_value == min(options.duration_in_traffic_value)].index[0]

In [156]:
chosen

'amb_10'

# For later (if we need to parse UI input)

In [ ]:
# Geocoding an address
# geocode_result = gmaps.geocode('1600 Amphitheatre Parkway, Mountain View, CA')